In [ ]:
import pandas as pd
import os 
import argparse
import math
from sklearn.model_selection import train_test_split, GridSearchCV,  StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import random as rd
from sklearn.metrics import matthews_corrcoef, f1_score
from pickle import dump
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2, l1
from keras.metrics import CategoricalAccuracy, AUC, BinaryCrossentropy, MeanSquaredError, TruePositives, FalsePositives, TrueNegatives,FalseNegatives, BinaryAccuracy,Precision,Recall,AUC
import tensorflow_addons as tfa
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import keras_tuner
from math import sqrt
from pickle import dump, load
import keras.backend as K

def matthews_correlation_coefficient(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

METRICS = [BinaryCrossentropy(name='Binary crossentropy'),  
      MeanSquaredError(name='Brier score'),
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'),
        matthews_correlation_coefficient
    # precision-recall curve
]

classes = ["anoxygenic_photoautotrophy_Fe_oxidizing",
"dark_sulfite_oxidation",
"knallgas_bacteria",
"oil_bioremediation",
"dark_sulfur_oxidation",
"dark_thiosulfate_oxidation",
"anoxygenic_photoautotrophy_S_oxidizing"
]
dataset = pd.read_csv('../Matrices/ko_df_validation.csv').set_index('Unnamed: 0')
validation_ds = pd.read_csv('../Matrices/ko_df_frag50.csv').set_index('Unnamed: 0')
validation_ds = validation_ds.sort_index()
functions = pd.read_csv("../Matrices/function_db.csv").set_index('Unnamed: 0')
def get_validation_set(to_validate, training_set):
    training_kos = training_set.columns
    validation_kos = to_validate.columns
    if len(set(training_kos).intersection(set(validation_kos))) == 0:
        print('no common kos between provided ones and training')
        return
    else:
        common = list(set(training_kos).intersection(set(validation_kos)))
        print('{} common kos'.format(len(common)))
        common_table = to_validate[common]
        #remove orthologs in validation not in the training
        to_remove = set(validation_kos) - set(training_kos)
        print('{} kos present in user set but not in training set will be removed'.format(len(to_remove)))
        missing = list(set(training_kos) - set(validation_kos))
        print('{} kos missing from the users et will be add to train the classifiers'.format(len(missing)))
        #missing_df = pd
        missed = pd.DataFrame(0, index = to_validate.index, columns= missing)
        to_submit = common_table.merge(missed, left_index = True, right_index = True)
        #print(list(validation_kos))
        to_submit = to_submit[list(training_kos)] #change order columns
        print('Shape of training dataset: {}, Shape of user dataset: {}'.format(training_set.shape, to_submit.shape))
        if list(to_submit.columns) != list(training_set.columns): 
            print('ERRORRRRR')
            return
    return to_submit

validation = get_validation_set(validation_ds, dataset)

2023-12-05 12:45:11.150015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-05 12:45:11.412135: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 12:45:11.464632: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-05 12:45:11.464642: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

Using TensorFlow backend


/home/baccus/.local/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
2023-12-05 12:45:13.304501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-05 12:45:13.304640: W tensorf

In [ ]:
dataset = pd.read_csv('../Matrices/genome_ko_all.csv').set_index('Genome')
validation_ds = pd.read_csv('../Matrices/ko_df_frag50.csv').set_index('Unnamed: 0')
validation_ds = validation_ds.drop(["GCF_000187005.1"])
validation_ds = validation_ds.sort_index()
functions = pd.read_csv("../Matrices/function_db.csv").set_index('Unnamed: 0')

In [13]:
functions_2 = pd.read_csv("../Matrices/predict_functions.csv").set_index('Unnamed: 0')
for classe in classes:
    model_load = "./validations/final_"+classe+".mdl_wts.hdf5"
    scaler = load(open('./validations/scaler_'+classe+'.sav', 'rb'))
    modelo = load_model(model_load, compile = True, custom_objects={"matthews_correlation_coefficient":matthews_correlation_coefficient})
    validation = get_validation_set(validation_ds, dataset)
    training_dataset_1 = validation.drop(columns=[validation.columns[0]])
    training_dataset_x = scaler.transform(training_dataset_1.values.reshape(training_dataset_1.shape[0],training_dataset_1.shape[1]))
    his = (modelo.predict(training_dataset_x) > 0.5).astype(np.int32)
    his2 = []
    results_TN = 0
    results_TP = 0
    results_FP = 0
    results_FN = 0
    for i in range(len(his)):
        
        if his[i] == 0:
            if functions_2[classe].values[i] == 0:
                results_TN += 1
            else:
                results_FN += 1
        else:
            if functions_2[classe].values[i] == 0:
                results_FP += 1
            else:
                results_TP += 1
        his2.append([his[i],functions_2[classe].values[i]])
        

    his2.append([results_TP, results_TN, results_FP, results_FN])
    his2 = pd.DataFrame(his2)
    names = "predict_"+classe+".csv"
    his2.to_csv(names)

9564 common kos
206 kos present in user set but not in training set will be removed
1906 kos missing from the users et will be add to train the classifiers
Shape of training dataset: (14363, 11470), Shape of user dataset: (4146, 11470)
130/130 [==============================] - 0s 2ms/step
9564 common kos
206 kos present in user set but not in training set will be removed
1906 kos missing from the users et will be add to train the classifiers
Shape of training dataset: (14363, 11470), Shape of user dataset: (4146, 11470)
130/130 [==============================] - 0s 2ms/step
9564 common kos
206 kos present in user set but not in training set will be removed
1906 kos missing from the users et will be add to train the classifiers
Shape of training dataset: (14363, 11470), Shape of user dataset: (4146, 11470)
130/130 [==============================] - 0s 2ms/step
9564 common kos
206 kos present in user set but not in training set will be removed
1906 kos missing from the users et will be 